# Analyzing.

## 1. Location.

In [1]:
from pymongo import MongoClient
import pandas as pd
import time
import importlib
import data.cleaning as clean
importlib.reload(clean)

<module 'data.cleaning' from 'D:\\ironhack\\proyectos\\GeoSpatialData_proy3\\data\\cleaning.py'>

Todo el proceso para descargar la base de datos Empresas de Mongo.

In [2]:
client = MongoClient("localhost:27017")

In [3]:
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [4]:
client.list_database_names()

['Ironhack', 'admin', 'config', 'local']

In [5]:
db = client["Ironhack"]

In [6]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Ironhack')

In [7]:
c = db.get_collection("companies")

Explore the data base to get the information I want for the company.

In [8]:
# This was to look if the data base from Mongo was well downloaded.
# c.find_one()

In [9]:
# This was to find with a specific condition of the ategory, and look for what i wanted to work on.
# list(c.find({}, {"category_code":1}))

#### Will choose the music industry for my company.
Getting the music buissnes companies and start analizing them.

In [10]:
# list of all the companies with category of music in all the world.
#list(c.find({"category_code":"music"}, {"name":1, "_id":0, "category_code":1}))

Get the companies Ubication COUNTRY.

In [11]:
def music_countries (category_code, country_code):
    condition1 = {"category_code":category_code}
    projection = {"name":1, "_id":0, "offices.country_code":country_code}
    countries = (list(c.find(condition1, 
               projection)))
    
    listcountry = [countries[i]["offices"][j]["country_code"] for i in range(len(countries)) for j in range(len(countries[i]["offices"]))]
    return listcountry
    

In [12]:
# list of the countries with the music company industry.
lstcountry = music_countries ("music", 1)

In [13]:
#countries[0]
#countries[0]["offices"]
#countries[0]["offices"][0]["country_code"]

In [14]:
# change it to a dictionary
country = {"country": lstcountry}

In [15]:
# make it a data frame
dfcountry = pd.DataFrame(country)

We see that for much USA is the place for a music company, I wanted to see if I could go to another country that had some type of competition between the USA, but if the company wants to grow, need to be in the USA.

In [16]:
dfcountry.value_counts()[:10]

country
USA        30
GBR         5
FRA         4
DEU         2
BEL         1
BRA         1
ESP         1
ITA         1
LUX         1
NLD         1
dtype: int64

hacer otra columna del value count de los paises

In [17]:
dfcountry.sample(3)

,country
22,NOR
37,USA
20,LUX
6,USA
15,USA


### Searching for the city

In [47]:
condition1 = {"category_code":"music"}
condition2 = {"offices.city":"San Diego"}
projection = {"name":1, "_id":0, "offices.address1":1}
x = (list(c.find({"$and":
                [condition1, condition2]},
                   projection)))

  

In [48]:
x

[{'name': 'Slacker',
  'offices': [{'address1': '16935 W. Bernardo Dr. Suite 101'}]},
 {'name': 'Bandsintown',
  'offices': [{'address1': '800 W Ivy Street, Unit C'},
   {'address1': '215 Lexington Ave'}]}]

In [18]:
def music_cities (category_code, cities):
    condition1 = {"category_code":category_code}
    projection = {"name":1, "_id":0, "offices.city":cities}
    city = (list(c.find(condition1, 
               projection)))
    
    lstcity = [city[i]["offices"][j]["city"] for i in range(len(city)) for j in range(len(city[i]["offices"]))]
    return lstcity

In [19]:
lstcity = music_cities ("music", 1)

Merge the 2 columns of country and city and new count column

In [20]:
def combine_lists (lst1, lst2):
    # change the 2 lists into a dictionary
    location = {"cities": lst1,
            "country": lst2} 
    # create them a Data Frame.
    location = pd.DataFrame(location)
    # create a new column with the value_counts of cities.
    location["count"] = location.groupby(["cities"])["country"].transform("count")
    return location

In [21]:
location_df = combine_lists (lstcity, lstcountry)

In [22]:
location_df.sample(3)

,cities,country,count
12,Berkeley,USA,1
8,Chelmsford,USA,1
23,Stockholm,SWE,1
45,London,GBR,4
1,New York,USA,6


Will do a drop duplicated and drop na 

In [23]:
def clean_df_citycountry (df):
    x = df.combine_first(df).reset_index().reindex(df.columns, axis=1).drop_duplicates()
    x.dropna(how="any", inplace=True)
    return x

In [24]:
location_clean = clean_df_citycountry (location_df)
location_clean.sample(3)

,cities,country,count
47,Rome,ITA,1
17,Somerville,USA,1
11,Oxford,GBR,1
34,Sao Paulo,BRA,1
6,Deerfield Beach,USA,1


Export the data frame of location into a CSV file.

In [25]:
def export_location (df):
    return df.to_csv(f"D:\\ironhack\\proyectos\\GeoSpatialData_proy3\csv\\location_clean.csv")
    

In [26]:
export_location (location_clean)

## 2. Startup environment.

In [39]:
def startupsdf (city, amount):
    # Conditions for city and amount of the company.
    condition1 = {"offices.city":city}
    condition2 = {"acquisitions.price_amount":{"$gte":amount}}
    projection = {"name":1, "_id":0, "category_code":1, "offices.address1":1}
    x = (list(c.find({"$and":
                             [condition1, condition2]},
                   projection)))
    # give a Data frame with conditions and address.
    df = pd.DataFrame(x)
    return df

In [43]:
startups = startupsdf ("San Diego", 1000000)
startups

,name,category_code,offices
0,Sony,hardware,[{'address1': '16530 Via Esprillo'}]
1,Qualcomm,semiconductor,[{'address1': '5775 Morehouse Drive'}]
2,Intuit,software,"[{'address1': '2632 Marine Way'}, {'address1':..."
3,Opera Software,software,"[{'address1': 'Waldemar Thranes gate 98'}, {'a..."
4,DivX,software,[{'address1': '4780 Eastgate Mall'}]
5,Accenture,consulting,"[{'address1': '1615 Murray Canyon Road'}, {'ad..."
6,Spreadtrum Communications,None,"[{'address1': '5960 Cornerstone Court,'}]"
7,Cypress Bioscience,biotech,[{'address1': '4350 Executive Drive'}]
8,ManTech,security,"[{'address1': '1100 New Jersey Ave, SE,Suite 7..."
9,Entropic Communications,semiconductor,[{'address1': '6290 Sequence Drive'}]


Export clean Data frame of startups, their category and their address inside San diego.

In [49]:
def export_SD_startups (df):
    return df.to_csv(f"D:\\ironhack\\proyectos\\GeoSpatialData_proy3\csv\\startups_sandiego.csv")

In [50]:
export_SD_startups (startups)